# Universidad Peruana de Ciencias Aplicadas
## Topicos de Ciencias de la Computacion - PC4
#### Estudiantes:
####  - Ibrahim Imanol Jordi Arquinigo Jacinto - U20191e650
####  - Ian Joaquin Sanchez Alva - U202124676
####  - Eduardo Jose Rivas Siesquen - U202216407
####  - Daniel Orlando Luis Lazaro - U202021900
####  Nov, 2025

In [1]:
import os
import random
import math
import asyncio
import nest_asyncio
import pickle 
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

# Parche Jupyter
nest_asyncio.apply()

# =============================================================================
# 1. GUI (PANEL REDISEÑADO PARA VER EL SCORE)
# =============================================================================
os.makedirs("static", exist_ok=True)

html_content = """
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Minecraft RL Hardcore</title>
    <style>
        body { font-family: 'Consolas', monospace; background: #0d0d0d; color: #eee; text-align: center; }
        canvas { background: #1b261b; border: 4px solid #3e2723; margin-top: 10px; box-shadow: 0 0 30px rgba(0,0,0,0.7); }
        
        /* PANEL DE CONTROL MEJORADO */
        .panel { 
            background: #212121; 
            padding: 15px; 
            display: inline-block; 
            border: 2px solid #616161; 
            border-radius: 10px; 
            margin-bottom: 5px; 
            min-width: 900px; /* Más ancho para que quepa todo */
            box-shadow: 0 4px 8px rgba(0,0,0,0.5);
        }
        
        .stat-box {
            display: inline-block;
            margin: 0 15px;
            text-align: center;
        }
        
        .stat-label { font-size: 12px; color: #aaa; display: block; }
        .stat-value { font-size: 18px; font-weight: bold; color: #fff; }
        
        /* Score destacado */
        .score-val { font-size: 22px; color: #00e676; text-shadow: 0 0 5px rgba(0, 230, 118, 0.5); }
        
        .legend { font-size: 12px; color: #aaa; margin-top: 5px; }
        input[type=range] { vertical-align: middle; accent-color: #d84315; }
    </style>
</head>
<body>
    <div class="panel">
        <div class="stat-box">
            <span class="stat-label">GEN</span>
            <span class="stat-value" id="gen">0</span>
        </div>
        <div class="stat-box">
            <span class="stat-label">DÍA</span>
            <span class="stat-value" id="dia">1</span>
        </div>
        <div class="stat-box">
            <span class="stat-label">AMENAZA</span>
            <span class="stat-value" id="threat" style="color:#ef5350">0</span>
        </div>
        <div class="stat-box">
            <span class="stat-label">VIVOS</span>
            <span class="stat-value" id="pob">0</span>
        </div>
        <div class="stat-box">
            <span class="stat-label">ZOMBIES</span>
            <span class="stat-value" id="zom">0</span>
        </div>
        
        <!-- PUNTUACIÓN DESTACADA -->
        <div class="stat-box" style="border-left: 1px solid #444; padding-left: 20px;">
            <span class="stat-label">PUNTAJE (PROMEDIO)</span>
            <span class="stat-value score-val" id="score">0</span>
        </div>

        <br><br>
        <label>⏩ Velocidad:</label> <input type="range" id="slider" min="1" max="50" value="1">
    </div>

    <canvas id="simCanvas" width="1000" height="700"></canvas>
    
    <div class="legend">
        🔴 Comida | 🟤 Madera | ⚪ Piedra | 
        <span style="color:#ff9800">🏠 Casa N1</span> -> <span style="color:#e65100">🏰 Fortaleza N5</span> | 
        <span style="color:#00e676">🟩 Barra HP</span>
    </div>

    <script>
        const canvas = document.getElementById('simCanvas');
        const ctx = canvas.getContext('2d');
        const ANCHO = 1000, ALTO = 700;

        function draw(data) {
            ctx.fillStyle = data.is_night ? "#0a0814" : "#2e7d32"; 
            ctx.fillRect(0, 0, ANCHO, ALTO);

            // Recursos
            data.recursos.forEach(r => {
                if(r.type === 'food') { 
                    ctx.fillStyle = '#ff1744'; ctx.beginPath(); ctx.arc(r.x, r.y, 4, 0, Math.PI*2); ctx.fill(); 
                } else {
                    ctx.fillStyle = (r.type === 'wood') ? '#795548' : '#bdbdbd'; 
                    ctx.fillRect(r.x-3, r.y-3, 7, 7);
                }
            });

            // Casas
            data.casas.forEach(h => {
                let size = 16 + (h.lvl * 6);
                if (h.lvl === 1) ctx.fillStyle = "#ff9800";
                else if (h.lvl === 2) ctx.fillStyle = "#e65100";
                else if (h.lvl === 3) ctx.fillStyle = "#d32f2f";
                else if (h.lvl === 4) ctx.fillStyle = "#7b1fa2";
                else ctx.fillStyle = "#212121";

                ctx.fillRect(h.x - size/2, h.y - size/2, size, size);
                
                // Barra HP
                ctx.fillStyle = "black"; ctx.fillRect(h.x - size/2, h.y - size/2 - 8, size, 5);
                let hp_pct = Math.max(0, h.hp / h.max_hp);
                let hp_color = hp_pct > 0.5 ? "#00e676" : (hp_pct > 0.25 ? "#ffea00" : "#ff1744");
                ctx.fillStyle = hp_color;
                ctx.fillRect(h.x - size/2, h.y - size/2 - 8, size * hp_pct, 5);
                
                // Texto Capacidad
                ctx.fillStyle = "white"; ctx.font = "12px monospace";
                ctx.fillText(h.occupants + "/" + h.cap, h.x - 10, h.y + 5);
            });

            // Steves
            data.steves.forEach(p => {
                ctx.beginPath();
                ctx.fillStyle = "#29b6f6"; 
                ctx.arc(p.x, p.y, 6, 0, Math.PI*2);
                ctx.fill();
                
                // Acción / Estado
                ctx.fillStyle = "white"; ctx.font = "10px Arial";
                if(p.action === "SLEEP") {
                    ctx.fillStyle = "#aa00ff"; 
                    ctx.fillText("💤", p.x - 5, p.y - 15);
                } else {
                    ctx.fillText(p.action, p.x - 10, p.y - 15);
                }

                ctx.fillStyle = "red"; ctx.fillRect(p.x-6, p.y-9, 12, 2);
                ctx.fillStyle = "#76ff03"; ctx.fillRect(p.x-6, p.y-9, 12 * (p.hp/100), 2);
            });

            // Zombies
            data.zombies.forEach(z => {
                ctx.beginPath();
                ctx.fillStyle = "#1b5e20"; 
                ctx.arc(z.x, z.y, 7 + (z.tier * 2), 0, Math.PI*2);
                ctx.fill();
                ctx.strokeStyle = "red"; ctx.lineWidth = 1; ctx.stroke();
            });

            document.getElementById('dia').innerText = data.dia;
            document.getElementById('threat').innerText = data.difficulty;
            document.getElementById('pob').innerText = data.steves.length;
            document.getElementById('zom').innerText = data.zombies.length;
            document.getElementById('gen').innerText = data.gen;
            
            // ACTUALIZAR SCORE VISIBLE
            let score = (data.avg_reward || 0).toFixed(1);
            let scoreEl = document.getElementById('score');
            scoreEl.innerText = score;
            scoreEl.style.color = data.avg_reward >= 0 ? "#00e676" : "#ff1744";
        }

        async function loop() {
            try {
                const response = await fetch('/data'); 
                if (response.ok) draw(await response.json());
            } catch(e) {}
            setTimeout(loop, 50); 
        }

        document.getElementById('slider').oninput = async function() {
            await fetch('/speed', { method: 'POST', body: JSON.stringify({val: this.value}) });
        };
        loop();
    </script>
</body>
</html>
"""
with open("static/index.html", "w", encoding='utf-8') as f: f.write(html_content)
print("✅ GUI (Score Visible) Generada.")

# =============================================================================
# 2. DEFINICIONES
# =============================================================================
ANCHO, ALTO = 1000, 700
DURACION_DIA = 250
DURACION_NOCHE = 400
CICLO = DURACION_DIA + DURACION_NOCHE

class Recurso:
    def __init__(self, t):
        self.x=random.randint(20,ANCHO-20); self.y=random.randint(20,ALTO-20); self.type=t

class Casa:
    def __init__(self, x, y, oid):
        self.x=x; self.y=y; self.owner_id=oid; 
        self.lvl=1; self.cap=1; self.occupants=0; 
        self.hp=1500; self.max_hp=1500 

# =============================================================================
# 3. CEREBRO Q-LEARNING
# =============================================================================

class QBrain:
    def __init__(self, parent_q_table=None):
        self.actions = ["IDLE", "EAT", "SLEEP", "GATHER_FOOD", "GATHER_MATS", "BUILD", "REPAIR", "FLEE", "ENTER_HOUSE"]
        self.lr = 0.1; self.gamma = 0.9; self.epsilon = 0.2 
        self.q_table = parent_q_table if parent_q_table else {}

    def get_state(self, agent, mgr):
        s_time = "NIGHT" if mgr.is_night else "DAY"
        min_d = 999
        for z in mgr.zombies:
            d = math.hypot(z.x - agent.x, z.y - agent.y)
            if d < min_d: min_d = d
        s_danger = "DANGER" if min_d < 150 else "SAFE"
        s_body = "DYING" if agent.hp < 40 else ("TIRED" if agent.energy < 40 else "OK")
        s_inv = "HAS_FOOD" if agent.food > 0 else ("HAS_MATS" if agent.wood>=3 else "POOR")
        s_house = "OUTSIDE"
        if agent.inside_house:
            s_house = "INSIDE" 
        return f"{s_time}_{s_danger}_{s_body}_{s_inv}_{s_house}"

    def choose_action(self, state):
        if "DYING" in state or "DANGER" in state:
            if state not in self.q_table: return "FLEE"
            return max(self.q_table[state], key=self.q_table[state].get)
        if random.uniform(0, 1) < self.epsilon: return random.choice(self.actions)
        if state not in self.q_table: self.q_table[state] = {a: 0.0 for a in self.actions}
        return max(self.q_table[state], key=self.q_table[state].get)

    def learn(self, state, action, reward, next_state):
        if state not in self.q_table: self.q_table[state] = {a: 0.0 for a in self.actions}
        if next_state not in self.q_table: self.q_table[next_state] = {a: 0.0 for a in self.actions}
        predict = self.q_table[state][action]
        target = reward + self.gamma * max(self.q_table[next_state].values())
        self.q_table[state][action] += self.lr * (target - predict)

# =============================================================================
# 4. AGENTE STEVE
# =============================================================================

class SteveRLAgent(Agent):
    def __init__(self, jid, password, parent_brain=None):
        super().__init__(jid, password)
        self.x = random.randint(50, ANCHO-50); self.y = random.randint(50, ALTO-50)
        self.hp = 100
        self.wood = 0; self.stone = 0; self.food = 0 
        self.energy = 60; self.inside_house = None 
        self.brain = QBrain(parent_brain.q_table.copy() if parent_brain else None)
        self.last_state = None; self.last_action = "IDLE"; self.current_action_name = "IDLE"
        self.decision_timer = 0; self.previous_hp = 100
        self.last_reward_val = 0 # Valor para la GUI

    async def setup(self): pass

    def perceive_and_act(self, mgr):
        if self.hp <= 0: return

        if mgr.tick % 30 == 0: self.energy -= 5 
        if self.energy <= 0: self.hp -= 5 
        
        damage_taken = self.hp < self.previous_hp
        self.previous_hp = self.hp
        if damage_taken: self.decision_timer = 0

        if self.decision_timer > 0:
            self.decision_timer -= 1
            self.execute_action(self.last_action, mgr, learning_mode=False) 
            return

        current_state = self.brain.get_state(self, mgr)
        reward = -1 
        
        if self.hp < 100 and self.last_action == "EAT": reward += 10 
        if self.energy <= 0: reward -= 50 
        
        if mgr.is_night:
            reward += 5 # Puntos por sobrevivir la noche
        
        if mgr.tick % 30 == 0: self.energy -= 5
        if self.energy <= 0: self.hp -= 5; reward -= 20
        
        # Guardar para la GUI
        self.last_reward_val = reward 

        if self.last_state: self.brain.learn(self.last_state, self.last_action, reward, current_state)

        action = self.brain.choose_action(current_state)
        
        if action in ["FLEE", "GATHER_FOOD", "GATHER_MATS", "ENTER_HOUSE"]: self.decision_timer = 50 
        elif action == "SLEEP": self.decision_timer = 30 
        else: self.decision_timer = 10

        self.last_state = current_state; self.last_action = action; self.current_action_name = action
        self.execute_action(action, mgr, learning_mode=True)

    def execute_action(self, action, mgr, learning_mode=False):
        reward_imm = 0
        if action == "IDLE": pass 
        elif action == "EAT":
            if self.food > 0:
                self.food -= 1; self.energy += 40; self.hp = min(100, self.hp + 20); reward_imm += 10 
            else: reward_imm -= 5
        elif action == "SLEEP":
            if self.inside_house and mgr.is_night:
                self.energy = min(100, self.energy + 2); self.hp = min(100, self.hp + 2); reward_imm += 25 
            else: reward_imm -= 10
        elif action == "GATHER_FOOD":
            c = self.get_closest_resource(mgr, 'food')
            if c:
                if self.move_to_target(c):
                    if c in mgr.resources:
                        mgr.resources.remove(c); self.food+=1; reward_imm += 5
            else: self.move_random()
        elif action == "GATHER_MATS":
            target = 'wood' if self.wood <= self.stone else 'stone'
            c = self.get_closest_resource(mgr, target)
            if c:
                if self.move_to_target(c):
                    if c in mgr.resources:
                        mgr.resources.remove(c)
                        if target=='wood': self.wood+=1
                        else: self.stone+=1
            else: self.move_random()
        elif action == "BUILD":
            if self.inside_house: 
                h = self.inside_house
                cost = h.lvl * 2 
                if self.wood >= cost and self.stone >= cost and h.lvl < 5:
                    self.wood -= cost; self.stone -= cost
                    h.lvl += 1; h.cap = h.lvl; h.max_hp += 500; h.hp = h.max_hp; reward_imm += 80 
                else: reward_imm -= 2 
            else: 
                if self.wood >= 3 and self.stone >= 3:
                    h = Casa(self.x, self.y, str(self.jid))
                    mgr.houses.append(h)
                    self.inside_house = h; h.occupants = 1
                    self.wood -= 3; self.stone -= 3; reward_imm += 80 
                else: reward_imm -= 2
        elif action == "REPAIR":
            if self.inside_house:
                h = self.inside_house
                if h.hp < h.max_hp:
                    if self.energy > 20:
                        self.energy -= 10; h.hp = min(h.max_hp, h.hp + 50); reward_imm += 30 
                    else: reward_imm -= 5 
                else: reward_imm -= 5 
            else: reward_imm -= 10
        elif action == "ENTER_HOUSE":
            closest_h = self.get_closest_house(mgr)
            if closest_h:
                if self.move_to_target(closest_h):
                    if closest_h.occupants < closest_h.cap:
                        self.inside_house = closest_h; closest_h.occupants += 1; reward_imm += 20 
            else: self.move_random()
        elif action == "FLEE":
            z = self.get_closest_zombie(mgr)
            if z:
                dx = self.x - z.x; dy = self.y - z.y
                self.move(dx, dy, 5.0); reward_imm += 5 

        if learning_mode and reward_imm != 0:
            self.last_reward_val += reward_imm # Sumar para mostrar en GUI
            self.brain.learn(self.last_state, action, reward_imm, self.last_state)

    def move_to_target(self, target):
        dist = math.hypot(target.x - self.x, target.y - self.y)
        if dist < 10: return True
        self.move(target.x - self.x, target.y - self.y, 3.5); return False
    def move_random(self): self.move(random.uniform(-1,1), random.uniform(-1,1), 1.5)
    def move(self, dx, dy, s):
        d = math.hypot(dx, dy)
        if d > 0: self.x += (dx/d)*s; self.y += (dy/d)*s
        self.x = max(0, min(ANCHO, self.x)); self.y = max(0, min(ALTO, self.y))
    def get_closest_resource(self, mgr, t):
        c=None; m=999
        for r in mgr.resources:
            if r.type == t:
                d = math.hypot(r.x-self.x, r.y-self.y)
                if d<m: m=d; c=r
        return c
    def get_closest_house(self, mgr):
        c=None; m=999
        for h in mgr.houses:
            d = math.hypot(h.x-self.x, h.y-self.y)
            if d<m: m=d; c=h
        return c
    def get_closest_zombie(self, mgr):
        c=None; m=999
        for z in mgr.zombies:
            d = math.hypot(z.x-self.x, z.y-self.y)
            if d<m: m=d; c=z
        return c

# =============================================================================
# 5. ZOMBIE (CAZADOR)
# =============================================================================

class ZombieObject:
    def __init__(self, difficulty):
        side = random.randint(0,3)
        if side==0: self.x, self.y = random.randint(0,ANCHO), 0
        else: self.x, self.y = 0, random.randint(0,ALTO)
        self.tier = 0; self.speed = 2.0 + (difficulty*0.3); self.damage = 1.0 + (difficulty*0.5)

    def logic(self, mgr):
        target = None; min_d = 999
        # Prioridad: Steve fuera
        for s in mgr.steves:
            if s.inside_house: continue 
            d = math.hypot(s.x - self.x, s.y - self.y)
            if d < min_d: min_d = d; target = s
        # Si no, casa ocupada
        if not target:
            for h in mgr.houses:
                if h.occupants > 0: 
                    d = math.hypot(h.x - self.x, h.y - self.y)
                    if d < min_d: min_d = d; target = h

        if target:
            dx = target.x - self.x; dy = target.y - self.y
            dist = math.hypot(dx, dy)
            if dist > 0: self.x += (dx/dist)*self.speed; self.y += (dy/dist)*self.speed
            
            if dist < 12:
                if isinstance(target, SteveRLAgent):
                    target.hp -= self.damage
                    target.brain.learn(target.last_state, target.last_action, -100, target.last_state)
                    if target.hp <= 0:
                        self.tier += 1; self.speed += 0.5; self.damage += 2; target.hp = 0
                elif isinstance(target, Casa):
                    target.hp -= self.damage * 0.5
                    if target.hp <= 0:
                        if target in mgr.houses: mgr.houses.remove(target)
                        for s in mgr.steves: 
                            if s.inside_house == target: 
                                s.inside_house = None
                                s.brain.learn(s.last_state, "IDLE", -100, s.last_state)

class ManagerAgent(Agent):
    async def setup(self):
        print("🎮 Servidor Q-Learning Iniciado.")
        self.steves = [SteveRLAgent(f"Gen0_{i}@loc", "pass") for i in range(25)]
        self.zombies = []
        self.houses = []
        self.resources = []
        self.respawn_resources()
        
        self.gen = 0; self.day = 1; self.tick = 0; self.difficulty = 0; self.delay = 0.05

        self.web.start(port=10000)
        self.web.app.router.add_static("/static", "static")
        self.web.add_get("/data", self.send_data, template=None)
        self.web.add_post("/speed", self.set_speed, template=None)
        self.add_behaviour(self.WorldLoop())

    def respawn_resources(self):
        self.resources = []
        for _ in range(120): self.resources.append(Recurso('food'))
        for _ in range(80): self.resources.append(Recurso('wood')) 
        for _ in range(80): self.resources.append(Recurso('stone'))

    async def send_data(self, req):
        avg_reward = 0
        if self.steves:
            avg_reward = sum(s.last_reward_val for s in self.steves) / len(self.steves)

        return {
            "gen": self.gen, "dia": self.day, "is_night": self.is_night, 
            "difficulty": self.difficulty, "avg_reward": avg_reward,
            "steves": [{"x":s.x,"y":s.y,"hp":s.hp,"action":s.current_action_name} for s in self.steves],
            "zombies": [{"x":z.x,"y":z.y,"tier":z.tier} for z in self.zombies],
            "recursos": [{"x":r.x,"y":r.y,"type":r.type} for r in self.resources],
            "casas": [{"x":h.x,"y":h.y,"hp":h.hp,"lvl":h.lvl,"cap":h.cap,"occupants":h.occupants,"max_hp":h.max_hp} for h in self.houses]
        }
    
    async def set_speed(self, req):
        self.delay = 0.05 / float((await req.json()).get('val',1))
        return {"status":"ok"}

    class WorldLoop(CyclicBehaviour):
        async def run(self):
            mgr = self.agent
            cycle = mgr.tick % CICLO
            mgr.is_night = cycle > DURACION_DIA
            
            if cycle == 0:
                mgr.day += 1
                mgr.zombies = [] 
                mgr.respawn_resources()
                if mgr.day % 2 == 0 and mgr.difficulty < 10: mgr.difficulty += 1 
                
                survivors = [s for s in mgr.steves if s.hp > 0]
                for s in survivors:
                    if s.inside_house and s.last_state:
                        s.brain.learn(s.last_state, s.last_action, 200, s.last_state)

                if survivors:
                    survivors.sort(key=lambda s: s.energy, reverse=True)
                    next_gen = []
                    for p in survivors: 
                        p.hp=100; p.energy=60; p.inside_house=None; p.x=random.randint(50,ANCHO-50)
                        next_gen.append(p)
                    
                    for p in survivors[:5]: 
                        if len(next_gen) < 20: 
                            child = SteveRLAgent(f"G{mgr.gen}_{len(next_gen)}@l","p", parent_brain=p.brain)
                            child.brain.epsilon = 0.3 
                            next_gen.append(child)
                    
                    mgr.steves = next_gen
                    mgr.gen += 1
                else:
                    print("💀 GAME OVER. Reinicio.")
                    mgr.steves = [SteveRLAgent(f"Adam_{i}@l","p") for i in range(25)]
                    mgr.respawn_resources()
                    mgr.gen = 0; mgr.difficulty = 0

            max_zom = 12 + (mgr.difficulty * 5)
            if mgr.is_night and len(mgr.zombies) < max_zom and random.random()<0.15:
                mgr.zombies.append(ZombieObject(mgr.difficulty))

            active_steves = []
            for s in mgr.steves:
                s.perceive_and_act(mgr)
                if s.hp > 0: active_steves.append(s)
            mgr.steves = active_steves
            
            for z in mgr.zombies: z.logic(mgr)

            mgr.tick += 1
            await asyncio.sleep(getattr(mgr, 'delay', 0.05))

async def main():
    manager = ManagerAgent("admin@localhost", "password")
    await manager.start()
    print("🚀 Minecraft Q-LEARNING FINAL Corriendo...")
    try:
        while True: await asyncio.sleep(1)
    except KeyboardInterrupt: await manager.stop()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running(): loop.create_task(main())
    else: asyncio.run(main())

✅ GUI (Score Visible) Generada.


🎮 Servidor Q-Learning Iniciado.
🚀 Minecraft Q-LEARNING FINAL Corriendo...
💀 GAME OVER. Reinicio.
💀 GAME OVER. Reinicio.
💀 GAME OVER. Reinicio.
💀 GAME OVER. Reinicio.
💀 GAME OVER. Reinicio.
💀 GAME OVER. Reinicio.
